In [1]:
import pandas as pd
df = pd.read_csv('school_attack.csv')
df

,school,image,tag_info
0,가거도초등학교,saved_images/image_83196.jpg,"{'total_tag': 4, 'total_label_tag': {'낙엽': 1, ..."
1,가경초등학교,saved_images/image_40152.jpg,"{'total_tag': 0, 'total_label_tag': {}}"
2,가거도초등학교,saved_images/image_83555.jpg,"{'total_tag': 5, 'total_label_tag': {'종이': 2, ..."
3,가거도초등학교,saved_images/image_93295.jpg,"{'total_tag': 1, 'total_label_tag': {'종이': 1}}"


In [2]:
import ast
df['tag_info'] = df['tag_info'].apply(ast.literal_eval)
df

,school,image,tag_info
0,가거도초등학교,saved_images/image_83196.jpg,"{'total_tag': 4, 'total_label_tag': {'낙엽': 1, ..."
1,가경초등학교,saved_images/image_40152.jpg,"{'total_tag': 0, 'total_label_tag': {}}"
2,가거도초등학교,saved_images/image_83555.jpg,"{'total_tag': 5, 'total_label_tag': {'종이': 2, ..."
3,가거도초등학교,saved_images/image_93295.jpg,"{'total_tag': 1, 'total_label_tag': {'종이': 1}}"


In [3]:
df['total_tag'] = df['tag_info'].apply(lambda x: x.get('total_tag', 0))
df['total_tag']

0    4
1    0
2    5
3    1
Name: total_tag, dtype: int64

In [ ]:
df['total_label_tag'] = df['tag_info'].apply(lambda x: x.get('total_label_tag', {}))
df['total_label_tag']

0    {'낙엽': 1, '담배꽁초': 2, '종이': 1}
1                               {}
2             {'종이': 2, '담배꽁초': 3}
3                        {'종이': 1}
Name: total_label_tag, dtype: object

In [5]:
# 모든 라벨 추출
all_labels = set()
for label_dict in df['total_label_tag']:
    all_labels.update(label_dict.keys())

# 각 라벨별 컬럼 생성
for label in all_labels:
    df[label] = df['total_label_tag'].apply(lambda x: x.get(label, 0))

df.head()

,school,image,tag_info,total_tag,total_label_tag,담배꽁초,낙엽,종이
0,가거도초등학교,saved_images/image_83196.jpg,"{'total_tag': 4, 'total_label_tag': {'낙엽': 1, ...",4,"{'낙엽': 1, '담배꽁초': 2, '종이': 1}",2,1,1
1,가경초등학교,saved_images/image_40152.jpg,"{'total_tag': 0, 'total_label_tag': {}}",0,{},0,0,0
2,가거도초등학교,saved_images/image_83555.jpg,"{'total_tag': 5, 'total_label_tag': {'종이': 2, ...",5,"{'종이': 2, '담배꽁초': 3}",3,0,2
3,가거도초등학교,saved_images/image_93295.jpg,"{'total_tag': 1, 'total_label_tag': {'종이': 1}}",1,{'종이': 1},0,0,1


In [6]:
df.drop(columns=['tag_info', 'total_label_tag'], inplace=True)
df

,school,image,total_tag,담배꽁초,낙엽,종이
0,가거도초등학교,saved_images/image_83196.jpg,4,2,1,1
1,가경초등학교,saved_images/image_40152.jpg,0,0,0,0
2,가거도초등학교,saved_images/image_83555.jpg,5,3,0,2
3,가거도초등학교,saved_images/image_93295.jpg,1,0,0,1
